In [ ]:
from library_dicom.dicom_processor.tools.pyradiomic import *
import SimpleITK as sitk 
import os 
import numpy as np 
import csv
import json 
from library_dicom.dicom_processor.model.csv_reader.CsvReader import CsvReader

In [ ]:
csv_path= '/media/deeplearning/LACIE SHARE/PVAB/NIFTI/unconstant/PVAB_NIFTI_unconstant.csv'
with open(csv_path, 'r') as csv_file :
    reader = csv.reader(csv_file, delimiter = ',') #liste pour chaque ligne 
    csv_file_ = []
    for row in reader :
        csv_file_.append(row)
        
del csv_file_[0] #enlever première ligne

In [ ]:
print("Nombre de mask à traiter : ", len(csv_file_))

#### - Keep only pet 0 

In [ ]:
pet0 = []
for row in csv_file_ : 
    if 'PET0' in row : 
        pet0.append(row)
print(len(pet0))

In [ ]:
#get date 
json_path = '/media/deeplearning/LACIE SHARE/PVAB/pvab_dataset.json'
dataset = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset.append(info)

#print(len(dataset))
for row in pet0 : 
    patient_id = row[0]
    for liste in dataset : 
        if patient_id in liste : 
            row.append(liste[-1])

#check if every study has its csv 
for row in pet0 : 
    if len(row) != 9 : 
        print(row)


In [ ]:
#get_date 
for row in pet0 : 
    #change csv_path 
    csv_file_path = row[-1]
    x = csv_file_path.replace('oncopole', 'deeplearning')
    row[-1] = x 

    #change pet and mask path 
    pet = row[5]
    mask= row[6]
    x = pet.replace('oncopole', 'deeplearning')
    y = mask.replace('oncopole', 'deeplearning')
    row[5] = x 
    row[6] = y 

    #get date 

    reader = CsvReader(row[-1])
    date = reader.get_date()
    row.append(date)


In [ ]:
for row in pet0 : 
    if len(row) != 10 : 
        print(row)

In [ ]:
for nifti in pet0: 
    try : 
        print(pet0.index(nifti))
        mask_path = nifti[-4]
        pet_path = nifti[-5]
        liste_center = get_center_of_mass(mask_path, thresh = True, pet_path = pet_path)
        d_max = calcul_distance_max(liste_center)
        nifti.append(d_max)
    except Exception as err : 
        print(nifti)
        print(err)

In [ ]:
nifti_directory = '/media/deeplearning/LACIE SHARE/PVAB'
filename = 'PVAB_dMax_v2.csv'


with open(os.path.join(nifti_directory, filename), 'w') as csv_file : 
    csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(["PATIENT ID", "DATE", "STUDY_UID", "D_MAX"])
    for row in pet0: 
        csv_writer.writerow([row[1], row[-2], row[2], row[-1]])
